In [67]:
import plotly.graph_objs as go
import plotly.express as px
import pandas as pd
import numpy as np
from plotly.subplots import make_subplots

# Update DataFrame

In [72]:
df = pd.read_csv("C:\\Users\\victo\\Documents\\TCC\\dev\\kotlin\\mq-base-api-kotlin\\untitled\\test_results\\receiver_latency.csv", sep = ';')
mq = "rabbitmq"

df = df[df.client == mq]

# Define utility functions

In [73]:
def sampleData(data):
    sample = data.sample(frac=1, replace=True)
    return {
        'avg_latency': sample.avg_latency.mean(), 
        'avg_throughput': sample.quantity.mean() * sample.message_size.mean() / 1000000.0,
        'balance': sample.balance.mean()
    }

def bootstrap(data):
    bts = [
        sampleData(data)
            for i in range(1000)
        ]
    btsDf = pd.DataFrame(bts)
    return {
        'avg_latency': btsDf.avg_latency.mean(), 
        'std_latency': btsDf.avg_latency.std(),
        'avg_throughput': btsDf.avg_throughput.mean(), 
        'std_throughput': btsDf.avg_throughput.std(),
        'avg_balance': btsDf.balance.mean(),
        'std_balance': btsDf.balance.std()
    }

def bootstrapLine(data, destinations, bench, clients, message_size):
    dataLine = data[
        (data['destinations'] == destinations) & 
        (data['benchmark'] == bench) & 
        (data['producers'] == clients) &
        (data['message_size'] == message_size)
    ]
    bts = bootstrap(dataLine)
    return {
        'benchmark': bench,
        'producers': clients,
        'consumers': clients,
        'destinations': destinations,
        'message_size': message_size,
        'average_latency': bts['avg_latency'],
        'std_latency': bts['std_latency'],
        'average_throughput': bts['avg_throughput'],
        'std_throughput': bts['std_throughput'],
        'avg_balance': bts['avg_balance'],
        'std_balance': bts['std_balance'],
    }
    

def bootstrapWholeData(data):
    dataBts = [
        bootstrapLine(data, 1, 'batch', 1, 500),
        bootstrapLine(data, 1, 'batch', 2, 500),
        bootstrapLine(data, 1, 'batch', 4, 500),
        bootstrapLine(data, 2, 'batch', 4, 500),
        bootstrapLine(data, 4, 'batch', 8, 500),
        bootstrapLine(data, 1, 'single', 1, 500),
        bootstrapLine(data, 1, 'single', 2, 500),
        bootstrapLine(data, 1, 'single', 4, 500),
        bootstrapLine(data, 2, 'single', 4, 500),
        bootstrapLine(data, 4, 'single', 8, 500),

        bootstrapLine(data, 1, 'batch', 2, 125),
        bootstrapLine(data, 1, 'batch', 2, 250),
        bootstrapLine(data, 1, 'batch', 2, 1000),
        bootstrapLine(data, 1, 'batch', 2, 2000),
        bootstrapLine(data, 1, 'batch', 2, 4000),
        bootstrapLine(data, 1, 'batch', 2, 8000),
        bootstrapLine(data, 1, 'batch', 2, 16000),
        bootstrapLine(data, 1, 'single', 2, 125),
        bootstrapLine(data, 1, 'single', 2, 250),
        bootstrapLine(data, 1, 'single', 2, 1000),
        bootstrapLine(data, 1, 'single', 2, 2000),
        bootstrapLine(data, 1, 'single', 2, 4000),
        bootstrapLine(data, 1, 'single', 2, 8000),
        bootstrapLine(data, 1, 'single', 2, 16000),
    ]
    return pd.DataFrame(dataBts)

def createErrorBox(fig, data, name, x, y, error_y, row = 1, col = 1, sy = False):
    fig.add_trace(
        go.Scatter(
            name=name,
            x=pd.to_numeric(data[x]),
            y=pd.to_numeric(data[y]),
            error_y=dict(type='data', array=data[error_y])
        ),
        row=row,
        col=col,
        secondary_y=sy
    )

def createErrorBox2(fig, data, name, x, y, error_x, row = 1, col = 1, sy = False):
    fig.add_trace(
        go.Scatter(
            name=name,
            x=pd.to_numeric(data[x]),
            y=pd.to_numeric(data[y]),
            error_x=dict(type='data', array=data[error_x])
        ),
        row=row,
        col=col,
        secondary_y=sy
    )

def createBox(fig, data, name, x, y, error_y, row = 1, col = 1):
    fig.add_trace(
        go.Scatter(
            name=name,
            x=pd.to_numeric(data[x]),
            y=pd.to_numeric(data[y]),
        ),
        row=row,
        col=col
    )

def createTimeseries(fig, data, color, name, y, row = 1, col = 1):
    fig.add_trace(
        go.Scatter(
            name=name,
            x=pd.to_numeric(data['normal_timestamp']),
            y=pd.to_numeric(data[y]),
            mode='lines',
            line=dict(color=color),
        ),
        row=row,
        col=col
    )

def suffixFn(bench):
    return ' para operações em massa' if bench == 'batch' else ' para operações unitárias'

# Bootstrap data

In [116]:
btsDf = bootstrapWholeData(df)

# Timeseries - Latency

In [117]:
for bench in ['single', 'batch']:
    d1 = df[df['destinations'] == 1 & (df['message_size'] == 500)]
    plot = d1[(d1['producers'] == 1) & (d1['benchmark'] == bench)]
    plot2 = d1[(d1['producers'] == 2) & (d1['benchmark'] == bench)]
    plot4 = d1[(d1['producers'] == 4) & (d1['benchmark'] == bench)]
    fig = make_subplots(rows=2, cols=1)

    # balance
    # avg_latency
    y = "avg_latency"

    # Latência
    # Balanceamento
    title = 'Latência'

    # Latência média (ms)
    y_title = "Latência média (ms)"

    # Tempo de execução normalizado
    x_title = "Tempo de execução normalizado"

    suffix = suffixFn(bench)

    createTimeseries(
        fig = fig,
        data = plot,
        color = 'rgba(200, 50, 0, 1)',
        name = "1, 1",
        y = y,
        row = 1    
    )

    createTimeseries(
        fig = fig,
        data = plot2,
        color = 'rgba(31, 200, 0, 1)',
        name = "2, 2",
        y = y,
        row = 1    
    )

    createTimeseries(
        fig = fig,
        data = plot4,
        color = 'rgba(31, 50, 200, 1)',
        name = "4, 4",
        y = y,
        row = 1    
    )

    d2 = df[(df['destinations'] == 2) & (df['benchmark'] == bench)]
    d4 = df[(df['destinations'] == 4) & (df['benchmark'] == bench)]

    createTimeseries(
        fig = fig,
        data = plot2,
        color = 'rgba(35, 164, 184, 1)',
        name = "1",
        y = y,
        row = 2
    )

    createTimeseries(
        fig = fig,
        data = d2,
        color = 'rgba(119, 40, 247, 1)',
        name = "2",
        y = y,
        row = 2
    )

    createTimeseries(
        fig = fig,
        data = d4,
        color = 'rgba(101, 168, 29, 1)',
        name = "4",
        y = y,
        row = 2
    )

    fig.update_xaxes(
        matches='x'
    )
    fig.update_layout(
        title={
            'text': title + suffix,
            'xanchor': 'center',
            'yanchor': 'top',
            'x':0.5,
            'y':0.85,
            'font': {
                'family': 'JetBrains Mono'
            }
        },
        xaxis={
            'title': {
                'text': x_title,
                'font': {
                    'family': 'JetBrains Mono'
                }
            },
            'anchor': 'free'
        },
        yaxis={
            'title': {
                'text': y_title,
                'font': {
                    'family': 'JetBrains Mono'
                },
            },
            'anchor': 'free',
        }
    )
    fig.write_image("images/" + mq + "/timeseries_" + bench + "_latency.pdf")
    fig.show()

# Compare - Latency/Balance/Throughput

In [118]:
for v, ev, t, l in [
    ['average_latency', 'std_latency', 'Latência média', 'Latência (ms)'], 
    ['avg_balance', 'std_balance', 'Balanceamento médio', 'Balanceamento'],
    ['average_throughput', 'std_throughput', 'Throughput médio', 'Throughput (MB)'],
    ]:
    for bench in ['batch', 'single']:
        fig = make_subplots(rows=1, cols=1, specs=[[{"secondary_y": True}]])
        batch = btsDf[(btsDf.benchmark == bench) & (btsDf.message_size == 500)]

        title = t + suffixFn(bench)

        createErrorBox2(
            fig = fig, 
            data = batch[batch.destinations == 1], 
            name = "Clientes",
            x = v, 
            y = "producers",
            error_x=ev,
            row = 1
        )
        createErrorBox2(
            fig = fig, 
            data = batch[(batch.destinations >= 2) | (batch.producers == 2)], 
            name = "Destinos", 
            x = v, 
            y = "destinations",
            error_x=ev,
            row = 1,
            sy=True
        )

        fig.update_yaxes(
            title_text="Número de <b>Clientes</b>", 
            secondary_y=False
        )
        fig.update_yaxes(
            title_text="Número de <b>Destinos</b>", 
            secondary_y=True
        )

        fig.update_layout(
            title={
                'text': title,
                'xanchor': 'center',
                'yanchor': 'top',
                'x':0.5,
                'y':0.85,
                'font': {
                    'family': 'JetBrains Mono'
                }
            },
            xaxis={
                'title': {
                    'text': l,
                    'font': {
                        'family': 'JetBrains Mono'
                    }
                },
                'anchor': 'free',
            }
        )

        fig.write_image("images/" + mq + "/comparison_" + bench + "_" + v + ".pdf")

        fig.show()

# Comparar ambos

In [121]:
for v, ev, t, l in [
    ['average_latency', 'std_latency', 'Latência média', 'Latência (ms)'], 
    ['avg_balance', 'std_balance', 'Balanceamento médio', 'Balanceamento'],
    ['average_throughput', 'std_throughput', 'Throughput médio', 'Throughput (MB)'],
    ]:
    fig = make_subplots(rows=1, cols=1, specs=[[{"secondary_y": True}]])
    for bench in ['batch', 'single']:
        batch = btsDf[(btsDf.benchmark == bench) & (btsDf.message_size == 500)]

        title = t
        ss = " - em Massa" if bench == 'batch' else " - Unitário"

        createErrorBox2(
            fig = fig, 
            data = batch[batch.destinations == 1], 
            name = "Clientes" + ss,
            x = v, 
            y = "producers",
            error_x=ev,
            row = 1
        )
        createErrorBox2(
            fig = fig, 
            data = batch[(batch.destinations >= 2) | (batch.producers == 2)], 
            name = "Destinos" + ss, 
            x = v, 
            y = "destinations",
            error_x=ev,
            row = 1,
            sy=True
        )

    fig.update_yaxes(
        title_text="Número de <b>Clientes</b>", 
        secondary_y=False
    )
    fig.update_yaxes(
        title_text="Número de <b>Destinos</b>", 
        secondary_y=True
    )

    fig.update_layout(
        title={
            'text': title,
            'xanchor': 'center',
            'yanchor': 'top',
            'x':0.5,
            'y':0.85,
            'font': {
                'family': 'JetBrains Mono'
            }
        },
        xaxis={
            'title': {
                'text': l,
                'font': {
                    'family': 'JetBrains Mono'
                }
            },
            'anchor': 'free',
        }
    )

    fig.write_image("images/" + mq + "/comparison_" + bench + "_" + v + "_all.pdf")

    fig.show()


# Compare message size

In [120]:
for v, ev, t, l in [
    ['average_latency', 'std_latency', 'Latência média', 'Latência (ms)'], 
    ['average_throughput', 'std_throughput', 'Throughput médio', 'Throughput (MB)'],
    ]:

    fig = make_subplots(rows=1, cols=1)

    bench = 'batch' 

    # producers
    # consumers
    # destinations
    # average_latency
    # std_latency
    # average_throughput
    # std_throughput
    # avg_balance

    # Throughput médio para operações em massa
    # Throughput (MB)

    # Balanceamento
    # Balanceamento

    title = t
    y = l

    yy = v
    yyr = ev

    createErrorBox(
        fig = fig, 
        data = btsDf[
        (btsDf.benchmark == 'batch') & 
        (btsDf.destinations == 1) &
        (btsDf.producers == 2)
    ].sort_values(by=['message_size']), 
        name = "Operações em Massa",
        x = "message_size", 
        y = yy,
        error_y=yyr,
        row = 1
    )

    createErrorBox(
        fig = fig, 
        data = btsDf[
        (btsDf.benchmark == 'single') & 
        (btsDf.destinations == 1) &
        (btsDf.producers == 2)
    ].sort_values(by=['message_size']), 
        name = "Operações Unitárias",
        x = "message_size", 
        y = yy,
        error_y=yyr,
        row = 1,
    )

    #fig.update_xaxes(type="log")
    if v != 'average_latency':
        fig.update_yaxes(type="log")

    fig.update_layout(
        title={
            'text': title,
            'xanchor': 'center',
            'yanchor': 'top',
            'x':0.5,
            'y':0.85,
            'font': {
                'family': 'JetBrains Mono'
            }
        },
        xaxis={
            'title': {
                'text': 'Tamanho da mensagem (bytes)',
                'font': {
                    'family': 'JetBrains Mono'
                }
            },
            'anchor': 'free',
        },
        yaxis={
            'title': {
                'text': y,
                'font': {
                    'family': 'JetBrains Mono'
                },
            },
            'anchor': 'free',
        }
    )

    fig.write_image("images/" + mq + "/comparison_message_size_" + v + ".pdf")

    fig.show()